Цель: изучить применение дерева решений в рамках задачи регрессии

В домашнем задании нужно решить задачу регрессии. В качестве датасета необходимо взять данные о недвижимости Калифорнии из библиотеки sklearn.datasets. Целевая переменная – MedHouseVal. Прочитать информацию о признаках датасета можно, выполнив следующий код – print(fetch_california_housing().DESCR). На полученных данных построить модель регрессии и дерево решений.




1.Получите данные и загрузите их в рабочую среду. (Jupyter Notebook или другую).

In [ ]:
from sklearn.datasets import fetch_california_housing
import pandas as pd
fetch = fetch_california_housing()
fetch.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])

In [ ]:
print(fetch.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

In [ ]:
df = pd.DataFrame(fetch.data, columns=fetch.feature_names)
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


2.Проведите первичный анализ.
a.Проверьте данные на пропуски. Удалите в случае обнаружения.
b.*Нормализуйте один из признаков.

In [ ]:
df.isna().sum()

MedInc        0
HouseAge      0
AveRooms      0
AveBedrms     0
Population    0
AveOccup      0
Latitude      0
Longitude     0
dtype: int64

In [ ]:
df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000


In [ ]:
# нормализирую Population
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaled = MinMaxScaler()
# scaled.fit(df[['Population']])
df['Population'] = scaled.fit_transform(df[['Population']])
df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,0.039869,3.070655,35.631861,-119.569704
std,1.899822,12.585558,2.474173,0.473911,0.031740,10.386050,2.135952,2.003532
min,0.499900,1.000000,0.846154,0.333333,0.000000,0.692308,32.540000,-124.350000
25%,2.563400,18.000000,4.440716,1.006079,0.021974,2.429741,33.930000,-121.800000
50%,3.534800,29.000000,5.229129,1.048780,0.032596,2.818116,34.260000,-118.490000
75%,4.743250,37.000000,6.052381,1.099526,0.048264,3.282261,37.710000,-118.010000
max,15.000100,52.000000,141.909091,34.066667,1.000000,1243.333333,41.950000,-114.310000


3.Разделите выборку на обучающее и тестовое подмножества. 80% данных оставить на обучающее множество, 20% - на тестовое.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, fetch.target, test_size = 0.2, random_state=7)

4.Обучите модель регрессии на обучающем множестве.

In [ ]:
from sklearn.linear_model import LinearRegression
model_linear = LinearRegression()
model_linear.fit(X_train,y_train)

LinearRegression()

5.Для тестового множества предскажите целевую переменную и сравните с истинным значением, посчитав точность предсказания модели. Для этого используйте встроенную функцию score.

In [ ]:
model_linear.score(X_test,y_test)

0.6075494500520069

6.Обучите дерево решений на обучающем множестве.
a. Повторите п. 5 для полученной модели.
b. Визуализируйте часть дерева решений. Убедитесь, что график получился читабельным. Посмотрите примеры визуализации по ссылке.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model_tree_reg = DecisionTreeRegressor(random_state=7).fit(X_train, y_train)
model_tree_reg.score(X_test, y_test) - 0.6075494500520069
# точность, при глубине дерева, не ограниченном, чуть упала


-0.008491861148345103

7.Оптимизируйте глубину дерева (max_depth). *Оптимизируйте ещё один параметр модели на выбор.
a. Повторите п. 5 для полученной модели.

In [ ]:
max_score, max_i = 0, 0
for i in range(1,15):

  model_tree_reg = DecisionTreeRegressor(max_depth=i, random_state=7).fit(X_train, y_train)
  score_ = model_tree_reg.score(X_test, y_test)
  if score_ > max_score:
    max_score = score_
    max_i = i
  # print(f'depth = {i}, score = {score_}')
print(f'!!!depth = {max_i}, score = {max_score}')
# оставляю 9
# проверю ещё метод:
max_score, max_m = 0, ''
for m in ['squared_error', 'friedman_mse', 'absolute_error', 'poisson']:
    model_tree_reg = DecisionTreeRegressor(criterion=m,max_depth=9, random_state=7).fit(X_train, y_train)
    score_ = model_tree_reg.score(X_test, y_test)
    if score_ > max_score:
      max_score = score_
      max_m = m
    print(f'm = {m}, s = {score_}')
print(max_m, max_score)
# по дефолту стоял лучший =) == squared_error

!!!depth = 9, score = 0.6935207936281012
m = squared_error, s = 0.6935207936281012
m = friedman_mse, s = 0.692680397958061
m = absolute_error, s = 0.6817220637325978
m = poisson, s = 0.11330582874054429
squared_error 0.6935207936281012


In [ ]:
# from sklearn.tree import DecisionTreeRegressor
model_tree_reg = DecisionTreeRegressor(max_depth=9, random_state=7).fit(X_train, y_train)
# model_tree_reg.score(X_test, y_test)

# from sklearn.tree import plot_tree
# import matplotlib.pyplot as plt
# plt.figure(figsize=(20,12))
# plot_tree(model_tree_reg,filled=True)


In [ ]:
model_tree_reg

DecisionTreeRegressor(max_depth=9, random_state=7)

In [ ]:
# from sklearn.externals.six import StringIO  
from six import StringIO
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(model_tree_reg, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

8.Сформулируйте выводы по проделанной работе.
a. Сравните точность двух моделей.
b. Напишите свое мнение, для каких задач предпочтительнее использовать обученные в работе модели? Какие у них есть плюсы и минусы?
Для получения зачета по этому домашнему заданию, должно быть как минимум реализовано обучение двух моделей, выведена их точность, оптимизирован один параметр дерева решений.

In [ ]:
0.6935207936281012 - 0.6075494500520069
# точность выросла на 8 процентов, по сравнению с обычной LinearRegression
'''
при линейно неразделимых данных возможно использовать дерево.
возможно перед началом работы с данными, для прикидки результирующих признаков 

'''

0.08597134357609426